# Data Preparation

National Centers for Environmental Information (NCEI) is responsible for hosting and providing access to one of the most significant archives on Earth, with comprehensive oceanic, atmospheric, and geophysical data. From the depths of the ocean to the surface of the sun and from million-year-old ice core records to near real-time satellite images, NCEI is the Nation’s leading authority for environmental information.

The Global Summary of the Month (GSOM) and Global Summary of the Year (GSOY) datasets consist of 55 climatological variables computed from summary of the day observations of the Global Historical Climatology Network Daily dataset. Of these, 53 are monthly and annual summary variables and two are season-to-date variables. 

This project consumes GSOY data. In order to keep volume of data under control, we only consider precipitation and temperature data for US. 

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Import necessary libraries

In [19]:
import os
import pandas as pd
from collections import OrderedDict
from sqlalchemy import create_engine, inspect
import tarfile

### Path to historic yearly weather information

In [4]:
RAW_DATA_FILE=os.path.join("..", "data", "gsoy-latest.tar.gz")

### Process archive downloaded from NCEI site - https://www.ncei.noaa.gov/data/gsoy/archive/

In [54]:
final_df = pd.DataFrame()

with tarfile.open(RAW_DATA_FILE, "r:gz") as tar:
    for index, member in enumerate(tar.getmembers()):
        # Extract only US data
        if not member.name.startswith('US'):
            continue
        csv = tar.extractfile(member)
        member_df = pd.read_csv(csv)
        # Skip records with no PRCP or TAVG. If record has TAVG, it also has TMAX and TMIN
        if 'TAVG' not in member_df.columns or 'PRCP' not in member_df.columns:
            continue
        
        # Extract only columns that are relevant for this analysis
        COLS_OF_INTEREST = ['DATE', 'NAME', 'STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'PRCP', 'TAVG', 'TMAX', 'TMIN']
        member_df = member_df[COLS_OF_INTEREST]
        
        # Use 1918 and later for analysis
        member_df = member_df[member_df.DATE >= 1918]
        final_df = pd.concat([final_df, member_df[COLS_OF_INTEREST]], sort=False)
        #if index > 10:
        #    break
        
final_df.head()

,DATE,NAME,STATION,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
0,2009,"SIOUX FALLS ENVIRON. CANADA, SD US",US009052008,43.7333,-96.6333,482.0,549.7,6.26,12.07,0.45
1,2010,"SIOUX FALLS ENVIRON. CANADA, SD US",US009052008,43.7333,-96.6333,482.0,755.1,7.32,12.91,1.72
2,2011,"SIOUX FALLS ENVIRON. CANADA, SD US",US009052008,43.7333,-96.6333,482.0,632.0,7.12,13.00,1.25
3,2012,"SIOUX FALLS ENVIRON. CANADA, SD US",US009052008,43.7333,-96.6333,482.0,NaN,9.75,16.33,3.17
4,2013,"SIOUX FALLS ENVIRON. CANADA, SD US",US009052008,43.7333,-96.6333,482.0,547.5,6.34,12.06,0.62


In [55]:
final_df.shape

(531240, 10)

In [56]:
final_df.describe()

,DATE,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
count,531240.000000,531240.000000,531240.000000,530432.000000,476620.000000,375051.000000,395099.000000,391517.000000
mean,1973.994349,39.514880,-98.702169,616.930986,860.332658,11.302478,17.847740,4.778901
std,27.122627,6.263603,17.678968,710.781196,504.028117,5.267375,5.477826,5.411927
min,1918.000000,19.061400,-177.350000,-59.100000,0.000000,-15.450000,-12.490000,-19.910000
25%,1953.000000,35.416670,-110.818900,146.300000,467.600000,7.620000,14.080000,0.970000
50%,1975.000000,39.668860,-96.716670,315.200000,834.200000,10.800000,17.380000,4.490000
75%,1998.000000,43.368060,-85.394100,878.400000,1145.900000,15.020000,21.870000,8.330000
max,2018.000000,71.321300,179.283330,13109.400000,9628.500000,26.970000,34.660000,24.270000


### Write to SQLite DB

In [57]:
final_df = final_df.reset_index().drop(labels='index', axis=1) # Remove duplicate indices
engine = create_engine('sqlite:///../db/adventurously.sqlite', echo=False)
final_df.to_sql('CLIMATE_HISTORY', if_exists='append', con=engine, index=True)

In [48]:
engine = create_engine('sqlite:///../db/adventurously.sqlite', echo=False)
pd.read_sql_query('select * from CLIMATE_HISTORY', con=engine)

,index,STATION,DATE,NAME,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
0,0,US009052008,2009,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,549.7,6.26,12.07,0.45
1,1,US009052008,2010,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,755.1,7.32,12.91,1.72
2,2,US009052008,2011,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,632.0,7.12,13.00,1.25
3,3,US009052008,2012,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,NaN,9.75,16.33,3.17
4,4,US009052008,2013,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,547.5,6.34,12.06,0.62
5,5,US009052008,2014,"SIOUX FALLS ENVIRON. CANADA, SD US",43.7333,-96.6333,482.0,NaN,NaN,NaN,NaN
6,6,USC00010063,1941,"ADDISON, AL US",34.2553,-87.1814,249.3,1186.4,NaN,NaN,NaN
7,7,USC00010063,1949,"ADDISON, AL US",34.2553,-87.1814,249.3,NaN,NaN,NaN,NaN
8,8,USC00010063,1986,"ADDISON, AL US",34.2553,-87.1814,249.3,1156.9,NaN,NaN,NaN
9,9,USC00010063,1987,"ADDISON, AL US",34.2553,-87.1814,249.3,1386.1,NaN,NaN,NaN


In [49]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.CLIMATE_HISTORY'>]
